In [1]:
import requests
import datetime
import json
import gzip
import pandas as pd
import numpy as np
import os
import time
import logging

from datetime import datetime
from bs4 import BeautifulSoup
from io import StringIO

In [2]:
url = 'https://smn.conagua.gob.mx/webservices/?method=1'
header = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 \
          (KHTML, like Gecko) Mobile/15E148'}

ruta_input = 'C:/Users/Usuario/Documents/data_municipios/'
ruta_tablas = 'C:/Users/Usuario/Documents/prueba_de/outputs/tablas/'
ruta_cruces = 'C:/Users/Usuario/Documents/prueba_de/outputs/tablas_cruzadas/'
ruta_current = 'C:/Users/Usuario/Documents/prueba_de/outputs/current/'
ruta_logs = 'C:/Users/Usuario/Documents/prueba_de/outputs/'

In [3]:
logging.basicConfig(filename = f'{ruta_logs}logs.log',format = '%(asctime)s %(message)s',filemode = 'a')

In [4]:
def getData(url_1, header_1):
    
    logger_scraping = logging.getLogger()
    
    try:
        result = requests.get(url_1, headers = header_1)
        data = gzip.decompress(result.content)
        data = json.loads(data)
        data = pd.DataFrame(data)
        data = data[['ides','idmun','prec','tmax','tmin']]

        data['ides'] = data['ides'].astype(int)
        data['idmun'] = data['idmun'].astype(int)
        data['prec'] = data['prec'].astype(float)
        data['tmax'] = data['tmax'].astype(float)
        data['tmin'] = data['tmin'].astype(float)
        
        logger_scraping.setLevel(logging.INFO)
        logger_scraping.info('Data extraída con éxito')

        return data
    
    except:
        logger_scraping.setLevel(logging.CRITICAL)
        logger_scraping.critical('Hubo un error al extraer o descomprimir la data')        

In [ ]:
n = 6
i = 0
count = 1

while i <= n:
    
    data = getData(url, header)
    data2 = pd.DataFrame()

    if count % 2 == 0:

        data2 = pd.concat([data, data2], ignore_index = True)
        data_final = data2.groupby(['ides', 'idmun'], as_index = False).mean()
        data_final.rename(columns = {'ides':'Cve_Ent', 'idmun':'Cve_Mun', 'prec':'prec_avg',
                                    'tmax':'tmax_avg', 'tmin':'tmin_avg'}, inplace = True)
        data_final['tavg'] = data_final[['tmax_avg','tmin_avg']].mean(axis = 1)

        now = datetime.now()
        now = now.strftime("%Y%m%d_%H%M")

        data_final.to_csv(f'{ruta_tablas}{now}.csv')
        
        logger_data_avg = logging.getLogger()
        logger_data_avg.setLevel(logging.INFO)
        logger_data_avg.info('Data con promedios creada con éxito')

        lista = os.listdir(ruta_input)
        lista_fechas = []

        for archivo in lista:

            anio = archivo[:4]
            mes = archivo[4:6]
            dia = archivo[6:8]

            date_archivo = pd.to_datetime(f'{anio}-{mes}-{dia}')
            date_archivo = date_archivo.date()

            lista_fechas.append(date_archivo)

        reciente = max(lista_fechas)

        id_reciente = int(str(reciente).split('-')[0] + str(reciente).split('-')[1] + str(reciente).split('-')[2])

        data_municipios = pd.read_csv(f'{ruta_input}/{id_reciente}/data.csv')
        data_municipios = data_municipios.merge(data_final, how = 'left')   
        data_municipios.fillna(0, inplace = True)

        data_municipios.to_csv(f'{ruta_cruces}/{now}.csv', index = False)
        data_municipios.to_csv(f'{ruta_current}/current.csv', index = False)
        
        logger_data_cruce = logging.getLogger()
        logger_data_cruce.setLevel(logging.INFO)
        logger_data_cruce.info('Data cruzada con éxito')

        del data, data2, data_final, data_municipios
        
        
        time.sleep(60)

    else: 
        data2 = pd.concat([data, data2], ignore_index = True)
        time.sleep(60)

    i += 1
    count += 1    